## 06 网格搜索和更多kNN中的超参数

In [1]:
import numpy as np
from sklearn import datasets

In [2]:
digits = datasets.load_digits()
X = digits.data
y = digits.target

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=666)

In [4]:
from sklearn.neighbors import KNeighborsClassifier

sk_knn_clf = KNeighborsClassifier(n_neighbors=4, weights="uniform")
sk_knn_clf.fit(X_train, y_train)
sk_knn_clf.score(X_test, y_test)

0.9916666666666667

### Grid Search
scikit learn中的超参数的网格搜索方法

In [16]:
# 所有想搜索的参数，键和参数的范围
# uniform的时候不需要对明可夫斯基距离进行搜索
param_grid = [
    {
        'weights': ['uniform'], 
        'n_neighbors': [i for i in range(1, 11)]
    },
    {
        'weights': ['distance'],
        'n_neighbors': [i for i in range(1, 11)], 
        'p': [i for i in range(1, 6)]
    },
    {
        'metric': ['chebyshev', 'seuclidean', 'seuclidean'],
        'n_neighbors': [i for i in range(1, 6)]
    }
]

In [6]:
knn_clf = KNeighborsClassifier()

In [7]:
# 网格搜索就是为了决定更好的模型，所以在model_selection中
from sklearn.model_selection import GridSearchCV

# 对哪一个分类器进行网格搜索，传入网格搜索相应的参数
grid_search = GridSearchCV(knn_clf, param_grid)

In [8]:
%%time
grid_search.fit(X_train, y_train)

Wall time: 1min 4s


GridSearchCV(cv=None, error_score=nan,
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='deprecated', n_jobs=None,
             param_grid=[{'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                          'weights': ['uniform']},
                         {'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                          'p': [1, 2, 3, 4, 5], 'weights': ['distance']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [9]:
# 返回最佳分类器的最佳参数
# 可能和手动网格搜索的参数不一样
# 因为使用了cross validation，比train-test-split准确度更高
grid_search.best_estimator_

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                     weights='uniform')

In [13]:
# 最佳分数
# 不是用户传入的参数，而是自己经过计算得到的参数在后面加上一个下划线
print(grid_search.best_score_)
print(grid_search.best_params_)

0.9860820751064653
{'n_neighbors': 1, 'weights': 'uniform'}


In [14]:
# 返回最佳参数的分类器
knn_clf = grid_search.best_estimator_
knn_clf.score(X_test, y_test)

0.9833333333333333

In [17]:
%%time
# n_jobs指定核数。-1是使用所有核数
# verbose是搜索中进行输出。整数越大信息越详细，一般2就OK
grid_search = GridSearchCV(knn_clf, param_grid, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)
print(grid_search.best_score_)
print(grid_search.best_params_)

Fitting 5 folds for each of 75 candidates, totalling 375 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 236 tasks      | elapsed:   11.0s
0.9860820751064653
{'n_neighbors': 1, 'weights': 'uniform'}
Wall time: 14.7 s
[Parallel(n_jobs=-1)]: Done 375 out of 375 | elapsed:   14.6s finished


### 其他超参数

metrics: [http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.DistanceMetric.html](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.DistanceMetric.html)